# Bond Pricing and Yields

## Learning Objectives

- Understand pricing conventions
- Learn about discount rates


## Using the Bond Class

In order to create a bond, we use the class Fixed Rate Bullet. We need to set at least 4 parameters. 

In [13]:
import pandas as pd
from pyfian.fixed_income.fixed_rate_bond import FixedRateBullet
import IPython.display as ipd  # rename to avoid conflict

# Define the bond
example_bond = FixedRateBullet(
    issue_dt="2025-01-01",
    maturity="2030-01-01",
    cpn=4,               # Coupon rate (%)
    cpn_freq=2,          # Semi-annual coupon
    settlement_date="2025-12-01",
    price=95             # Price input
)

# Build table with only inputs
inputs_data = [
    {"Attribute": "Instrument", "Value": example_bond.__class__.__name__},
    {"Attribute": "Issue Date", "Value": example_bond.issue_dt},
    {"Attribute": "Maturity Date", "Value": example_bond.maturity},
    {"Attribute": "Settlement Date", "Value": example_bond.get_settlement_date()},
    {"Attribute": "Notional", "Value": example_bond.notional},
    {"Attribute": "Coupon (%)", "Value": example_bond.cpn},
    {"Attribute": "Coupon Frequency", "Value": example_bond.cpn_freq},
    {"Attribute": "Day Count Convention", "Value": example_bond.day_count_convention.__class__.__name__},
    {"Attribute": "Price", "Value": example_bond.get_price()},
]

df_inputs = pd.DataFrame(inputs_data)

# Display table without index
ipd.display(df_inputs.style.hide(axis="index"))


Attribute,Value
Instrument,FixedRateBullet
Issue Date,2025-01-01 00:00:00
Maturity Date,2030-01-01 00:00:00
Settlement Date,2025-12-01 00:00:00
Notional,100
Coupon (%),4
Coupon Frequency,2
Day Count Convention,DayCountActualActualBond
Price,95


This bond will have the following cash flows:

In [15]:
import pandas as pd
from pyfian.fixed_income.fixed_rate_bond import FixedRateBullet
import IPython.display as ipd  # rename to avoid conflicts

# Define the bond
example_bond = FixedRateBullet(
    issue_dt="2025-01-01",
    maturity="2030-01-01",
    cpn=4,
    cpn_freq=2,
    settlement_date="2025-12-01",
    price=95
)

# Get cash flows for a specific settlement date
cash_flows = example_bond.cash_flows(settlement_date="2026-02-01")

# Convert to DataFrame
df2 = pd.DataFrame(cash_flows, columns=["Cash Flows"])

# Display without index
ipd.display(df2.style.hide(axis="index"))


Cash Flows
2.000000
2.000000
2.000000
2.000000
2.000000
2.000000
2.000000
102.000000


## Discount, Par & Premium

Bonds can be sold at a discount, at par, or at a premium. When a bond is sold at a discount, the difference between its face value and the lower purchase price serves as additional compensation for the investor, reflecting a yield that exceeds the coupon rate. Conversely, when a bond is sold at par, the yield equals the coupon rate. When sold at a premium, the bond's coupon rate exceeds the yield, and investors pay more upfront in exchange for higher coupon payments. 

## Quoting the price of a bond

### Clean Price

The prices that we see in the news is typically quoted as the clean price. The clean price is calculated as: 

$$\text{Clean Price} = \text{Dirty Price} - \text{Accrued Interest}$$

### Dirty Price

The dirty price includes the accrued interest. 

$$\text{Dirty Price} = \text{Clean Price} + \text{Accrued Interest}$$

### Accrued Interest

Since interest is paid out just several times a year, we need to compute the interest that has accrued since the last coupon payment. The formula for accrued interest is: 


$$\text{Accrued Interest} = \frac{\text{Coupon Rate} \times \text{Face Value} \times \text{Days Accrued}}{\text{Days in Coupon Period}}$$

## Discrete Periods vs Continuous compounding

We can compound interest using either discrete periods or continuous compounding. For instance, when calculating accrued interest between two periods, we can use the following formulas for computing the present or future value of the instrument. 

The formula for present value of continuous compounding interest is: $\text{PV}= \text{FV} e^{-rt}$.

The formula for future value of continuous compounding interest is: $\text{FV} = \text{PV} e^{rt}$.

## Introduction to Yield Concepts

### Current Yield

The current yield is the ratio of the annual coupon divided by the current price.

### Yield to Maturity (YTM)
The YTM is the total return anticipated if the bond is held to maturity (conceptual introduction; details in future classes).



## Quantifying return of a bond

### IRR (Internal Rate of Return)


IRR is the discount rate that makes the net present value (NPV) of a series of cash flows equal to zero. When a bond is held to maturity, it is equal to the YTM.


### XIRR: Non periodic IRR

XIRR, or Extended Internal Rate of Return, is a financial metric used to calculate the annualized rate of return for a series of cash flows that occur at irregular intervals.

## Calculating yields and prices

Here we calculate the different variables of the bond using the FixedRateBullet class. 

In [ ]:
import pandas as pd
from pyfian.fixed_income.fixed_rate_bond import FixedRateBullet
import IPython.display as ipd  # to avoid conflict
from pyfian.time_value.irr import irr
# Define the bond
example_bond = FixedRateBullet(
    issue_dt="2025-01-01",
    maturity="2030-01-01",
    cpn=4,               # Coupon rate (%)
    cpn_freq=2,          # Semi-annual coupon
    settlement_date="2025-12-01",
    price=95,
    notional = 100,
)

# Calculate outputs
clean_price = example_bond.get_price()
accrued_interest = example_bond.accrued_interest()
dirty_price = clean_price + accrued_interest
ytm = example_bond.get_yield_to_maturity()
current_yield = example_bond.cpn/clean_price
irr = 0000    # IRR based on cash flows
xirr = 0000    # XIRR (time-weighted IRR)

# Build table with inputs and calculated values
data = [
    # Inputs
    {"Attribute": "Instrument", "Value": example_bond.__class__.__name__, "Category": ""},
    {"Attribute": "Issue Date", "Value": example_bond.issue_dt, "Category": ""},
    {"Attribute": "Maturity Date", "Value": example_bond.maturity, "Category": ""},
    {"Attribute": "Settlement Date", "Value": example_bond.get_settlement_date(), "Category": ""},
    {"Attribute": "Notional", "Value": example_bond.notional, "Category": ""},
    {"Attribute": "Coupon (%)", "Value": example_bond.cpn, "Category": ""},
    {"Attribute": "Coupon Frequency", "Value": example_bond.cpn_freq, "Category": ""},
    {"Attribute": "Day Count Convention", "Value": example_bond.day_count_convention.__class__.__name__, "Category": ""},
    {"Attribute": "Price (Input)", "Value": example_bond.get_price(), "Category": ""},
    
    # Calculated
    {"Attribute": "Clean Price", "Value": f"{clean_price:.2f}", "Category": "★"},
    {"Attribute": "Accrued Interest", "Value": f"{accrued_interest:.2f}", "Category": "★"},
    {"Attribute": "Dirty Price", "Value": f"{dirty_price:.2f}", "Category": "★"},
    {"Attribute": "Yield to Maturity (%)", "Value": f"{ytm * 100:.2f}", "Category": "★"},
    {"Attribute": "Current Yield (%)", "Value": f"{current_yield * 100:.2f}", "Category": "★"},
    {"Attribute": "IRR (%)", "Value": f"{irr * 100:.2f}", "Category": "★"},
    {"Attribute": "XIRR (%)", "Value": f"{xirr * 100:.2f}", "Category": "★"},
]

# Convert to DataFrame
df = pd.DataFrame(data)

# Move Category to last column
df = df[["Attribute", "Value", "Category"]]

# Display table without index
ipd.display(df.style.hide(axis="index"))


Attribute,Value,Category
Instrument,FixedRateBullet,
Issue Date,2025-01-01 00:00:00,
Maturity Date,2030-01-01 00:00:00,
Settlement Date,2025-12-01 00:00:00,
Notional,100,
Coupon (%),4,
Coupon Frequency,2,
Day Count Convention,DayCountActualActualBond,
Price (Input),95,
Accrued Interest,1.66,★
